# QC Research API Method Testing

This notebook systematically tests different methods to read backtest data in QuantConnect Research.
Run each cell and note which ones succeed.

In [ ]:
# Initialize QuantBook
from QuantConnect import *
from QuantConnect.Research import *
import pandas as pd
import json

qb = QuantBook()
print(f'QuantBook initialized')
print(f'Project ID: {qb.project_id}')

In [ ]:
# Test data
project_id = qb.project_id
backtest_id = '67dd62a13c9acfba69bb3493'

print(f'Testing with:')
print(f'  Project ID: {project_id}')
print(f'  Backtest ID: {backtest_id}')

## Test 1: qb.ReadBacktest (PascalCase)

In [ ]:
try:
    print('Testing: qb.ReadBacktest()')
    backtest = qb.ReadBacktest(project_id, backtest_id)
    print('SUCCESS: qb.ReadBacktest() works!')
    print(f'  Type: {type(backtest)}')
    print(f'  Attributes: {dir(backtest)[:10]}')
except Exception as e:
    print(f'FAILED: {type(e).__name__}: {e}')

## Test 2: qb.read_backtest (snake_case)

In [ ]:
try:
    print('Testing: qb.read_backtest()')
    backtest = qb.read_backtest(project_id, backtest_id)
    print('SUCCESS: qb.read_backtest() works!')
    print(f'  Type: {type(backtest)}')
    print(f'  Attributes: {dir(backtest)[:10]}')
except Exception as e:
    print(f'FAILED: {type(e).__name__}: {e}')

## Test 3: api.read_backtest (Api object)

In [ ]:
try:
    print('Testing: api.read_backtest()')
    from QuantConnect.Api import Api
    api = Api()
    backtest = api.read_backtest(project_id, backtest_id)
    print('SUCCESS: api.read_backtest() works!')
    print(f'  Type: {type(backtest)}')
    print(f'  Attributes: {dir(backtest)[:10]}')
except Exception as e:
    print(f'FAILED: {type(e).__name__}: {e}')

## Test 4: api.ReadBacktest (PascalCase)

In [ ]:
try:
    print('Testing: api.ReadBacktest()')
    from QuantConnect.Api import Api
    api = Api()
    backtest = api.ReadBacktest(project_id, backtest_id)
    print('SUCCESS: api.ReadBacktest() works!')
    print(f'  Type: {type(backtest)}')
    print(f'  Attributes: {dir(backtest)[:10]}')
except Exception as e:
    print(f'FAILED: {type(e).__name__}: {e}')

## Test 5: Inspect QuantBook methods

In [ ]:
print('All QuantBook methods containing "backtest" (case-insensitive):')
backtest_methods = [m for m in dir(qb) if 'backtest' in m.lower()]
for method in backtest_methods:
    print(f'  - qb.{method}')

## Test 6: Inspect Api methods

In [ ]:
from QuantConnect.Api import Api
api = Api()

print('All Api methods containing "backtest" (case-insensitive):')
backtest_methods = [m for m in dir(api) if 'backtest' in m.lower()]
for method in backtest_methods:
    print(f'  - api.{method}')

## Test 7: Try successful method to access data

In [ ]:
# Use whichever method succeeded above
# Try multiple approaches

methods_to_try = [
    ('qb.ReadBacktest', lambda: qb.ReadBacktest(project_id, backtest_id)),
    ('qb.read_backtest', lambda: qb.read_backtest(project_id, backtest_id)),
    ('api.read_backtest', lambda: api.read_backtest(project_id, backtest_id)),
    ('api.ReadBacktest', lambda: api.ReadBacktest(project_id, backtest_id)),
]

backtest = None
working_method = None

for method_name, method_func in methods_to_try:
    try:
        print(f'Trying {method_name}...')
        backtest = method_func()
        working_method = method_name
        print(f'  SUCCESS with {method_name}')
        break
    except Exception as e:
        print(f'  Failed: {type(e).__name__}')

if backtest:
    print(f'\nWorking method: {working_method}')
    print(f'Backtest object type: {type(backtest)}')
else:
    print('\nNo method worked!')

## Test 8: Access Charts (test different case variations)

In [ ]:
if backtest:
    print('Testing chart access...')
    
    # Test Charts (PascalCase)
    try:
        charts = backtest.Charts
        print('SUCCESS: backtest.Charts (PascalCase)')
        print(f'  Available charts: {list(charts.keys())[:5]}')
    except Exception as e:
        print(f'FAILED: backtest.Charts - {type(e).__name__}')
    
    # Test charts (snake_case)
    try:
        charts = backtest.charts
        print('SUCCESS: backtest.charts (snake_case)')
        print(f'  Available charts: {list(charts.keys())[:5]}')
    except Exception as e:
        print(f'FAILED: backtest.charts - {type(e).__name__}')
else:
    print('No backtest object to test')

## Test 9: Extract Equity Curve

In [ ]:
if backtest:
    print('Testing equity curve extraction...')
    
    # Try PascalCase
    try:
        charts = backtest.Charts
        equity_chart = charts['Strategy Equity']
        equity_series = equity_chart.Series['Equity']
        equity_values = [p.y for p in equity_series.Values]
        print('SUCCESS: PascalCase (Charts/Series/Values)')
        print(f'  Extracted {len(equity_values)} equity points')
        returns = pd.Series(equity_values).pct_change().dropna()
        print(f'  Calculated {len(returns)} returns')
    except Exception as e:
        print(f'FAILED PascalCase: {type(e).__name__}: {e}')
    
    # Try snake_case
    try:
        charts = backtest.charts
        equity_chart = charts['Strategy Equity']
        equity_series = equity_chart.series['Equity']
        equity_values = [p.y for p in equity_series.values]
        print('SUCCESS: snake_case (charts/series/values)')
        print(f'  Extracted {len(equity_values)} equity points')
        returns = pd.Series(equity_values).pct_change().dropna()
        print(f'  Calculated {len(returns)} returns')
    except Exception as e:
        print(f'FAILED snake_case: {type(e).__name__}: {e}')
else:
    print('No backtest object to test')

## Test 10: Access Statistics

In [ ]:
if backtest:
    print('Testing statistics access...')
    
    # Try Statistics (PascalCase)
    try:
        stats = backtest.Statistics
        sharpe = stats.get('Sharpe Ratio', 'N/A')
        print('SUCCESS: backtest.Statistics (PascalCase)')
        print(f'  Sharpe Ratio: {sharpe}')
        print(f'  Available stats: {list(stats.keys())[:5]}')
    except Exception as e:
        print(f'FAILED: backtest.Statistics - {type(e).__name__}')
    
    # Try statistics (snake_case)
    try:
        stats = backtest.statistics
        sharpe = stats.get('Sharpe Ratio', 'N/A')
        print('SUCCESS: backtest.statistics (snake_case)')
        print(f'  Sharpe Ratio: {sharpe}')
        print(f'  Available stats: {list(stats.keys())[:5]}')
    except Exception as e:
        print(f'FAILED: backtest.statistics - {type(e).__name__}')
else:
    print('No backtest object to test')

## Summary: Copy Working Pattern Below

In [ ]:
print('='*60)
print('SUMMARY: Working API Pattern')
print('='*60)
print('Copy the code from whichever tests succeeded above.')
print('This is the pattern to use in research.ipynb')
print('='*60)